In [ ]:
import math
import pandas as pd

# ----------------------------
# 1. Base cost config: ONE DAY
# ----------------------------

models_per_day = {
    "Ultra_Light": {
        "Breakfast": 12,
        "Lunch": 20,
        "Dinner": 20,
        "Snacks": 5,
        "Tea/Coffee/Milk": 5,
        "Staff": 8,
        "Packing": 2,
        "Fuel": 8,
        "Delivery": 2,
        "Overhead": 2
    },
    "Light": {
        "Breakfast": 30,
        "Lunch": 65,
        "Dinner": 60,
        "Snacks": 15,
        "Tea/Coffee/Milk": 10,
        "Staff": 20,
        "Packing": 10,
        "Fuel": 10,
        "Delivery": 8,
        "Overhead": 8
    },
    "Basic": {
        "Breakfast": 60,
        "Lunch": 140,
        "Dinner": 130,
        "Snacks": 20,
        "Tea/Coffee/Milk": 15,
        "Staff": 50,
        "Packing": 25,
        "Fuel": 25,
        "Delivery": 20,
        "Overhead": 20
    },
    "Premium": {
        "Breakfast": 90,
        "Lunch": 220,
        "Dinner": 200,
        "Snacks": 50,
        "Tea/Coffee/Milk": 25,
        "Staff": 100,
        "Packing": 40,
        "Fuel": 40,
        "Delivery": 30,
        "Overhead": 30
    },
    "Ultra_Premium": {
        "Breakfast": 150,
        "Lunch": 400,
        "Dinner": 375,
        "Snacks": 90,
        "Tea/Coffee/Milk": 40,
        "Staff": 180,
        "Packing": 80,
        "Fuel": 70,
        "Delivery": 60,
        "Overhead": 50
    }
}

linear_categories = ["Breakfast", "Lunch", "Dinner", "Snacks", "Tea/Coffee/Milk"]
sublinear_exponents = {
    "Staff": 0.80,
    "Packing": 0.85,
    "Fuel": 0.90,
    "Delivery": 0.80,
    "Overhead": 0.80
}

# ----------------------------
# 2. Cost calculation function
# ----------------------------

def compute_one_day_costs(per_day_base, people):
    costs = {}
    for cat, per_person_val in per_day_base.items():
        if cat in linear_categories:
            costs[cat] = int(round(per_person_val * people))
        else:
            exp = sublinear_exponents.get(cat, 1.0)
            costs[cat] = int(round(per_person_val * (people ** exp)))
    total = sum(costs.values())
    avg_per_person_1_day = total / people if people > 0 else 0
    costs["People"] = people
    costs["Total"] = int(round(total))
    costs["Avg_Per_Person_1_day"] = round(avg_per_person_1_day, 2)
    return costs

# ----------------------------
# 3. Generate rows for all models
# ----------------------------

people_custom = [1] + list(range(25, 1001, 25))
rows = []
for model_name, per_day_base in models_per_day.items():
    for p in sorted(set(people_custom)):
        costs = compute_one_day_costs(per_day_base, p)
        row = {"Model": model_name, **costs}
        rows.append(row)

df = pd.DataFrame(rows)

# Reorder columns
base_keys = list(models_per_day["Basic"].keys())
cols_order = ["Model", "People"] + base_keys + ["Total", "Avg_Per_Person_1_day"]
df = df[cols_order]

# ----------------------------
# 4. Add VAT and Grand Total
# ----------------------------

df["VAT_5pct"] = (df["Total"] * 0.05).round(2)
df["Grand_Total"] = (df["Total"] + df["VAT_5pct"]).round(2)

# ----------------------------
# 5. Add ₹ in headers (not in cell values)
# ----------------------------

def add_currency_to_headers(df, cols, symbol="₹"):
    return [f"{col} ({symbol})" if col in cols else col for col in df.columns]

currency_cols = base_keys + ["Total", "Avg_Per_Person_1_day", "VAT_5pct", "Grand_Total"]
df.columns = add_currency_to_headers(df, currency_cols)

# ----------------------------
# 6. Save to CSV
# ----------------------------

filename = "kitchen_pooling_model.csv"
df.to_csv(filename, index=False, encoding="utf-8-sig")

print(f"✅ Saved: {filename}")

✅ Saved: indian_kitchen_pooling_model_per_day.csv
